This notebook shows the methods to convert raster to points using GDAL. The outputs are saved in a 'csv' format along with coordinates. The output file will be located in the same directory the input file is. It will be the same name as the input file.

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
from pyproj import Transformer
from osgeo import gdal
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [ ]:
# Define the working directory
os.chdir('YOUR WORKING DIRECTORY')

In [ ]:
def raster_to_points(raster_path):
    
    # Open the band image using gdal
    gdal_raster = gdal.Open(raster_path)
    
    # Convert raster gdal to XYZ points
    point_XYZ = gdal.Translate(raster_path.split('.')[0]+'.xyz',
                               gdal_raster,
                               format='XYZ', creationOptions=["ADD_HEADER_LINE=YES", "DECIMAL_PRECISION=20"])
    
    # Release the files
    point_XYZ = None
    gdal_raster = None
    
    # Rename the xyz file to csv
    try:
        os.rename(raster_path.split('.')[0]+'.xyz', raster_path.split('.')[0]+'.csv')
    except:
        os.remove(raster_path.split('.')[0]+'.csv')
        os.rename(raster_path.split('.')[0]+'.xyz', raster_path.split('.')[0]+'.csv')
    
    # Read the csv as pandas dataframe
    point = pd.read_csv(raster_path.split('.')[0]+'.csv', delimiter=' ')
    
    # Remove nodata with 0.0
    point = point[point['Z']!=0.0]
    
    # Remove the csv file from the disk
    os.remove(raster_path.split('.')[0]+'.csv')
    
    # Get only the Lon and Lat
    point = point[['X', 'Y']]
    point.columns = ['X_r', 'Y_r']
    point.reset_index(drop=True, inplace=True)
    
    # Insert new columns for X and Y
    point.insert(2, "Lat_r", 0)
    point.insert(3, "Lon_r", 0)

    # Convert lat lon to X and Y in UTM 15N
    # (Our study area is in UTM 15N, Change if necessary)
    # Find the EPSG code for your study area
    transformer = Transformer.from_crs(32615, 4326)
    Lat_r, Lon_r = transformer.transform(point.iloc[:, 0], point.iloc[:, 1])
    point.loc[:, 'Lat_r'] = Lat_r
    point.loc[:, 'Lon_r'] = Lon_r
    
    # Save the updated csv
    point.to_csv(raster_path.split('.')[0]+'.csv')
    
    print(os.path.basename(raster_path) + " done!")

In [ ]:
# Define the input data path
img_path = './raw_0_rd_rf_or_pr_warp.dat'

In [ ]:
%%time
raster_to_points(img_path)